In [1]:
import json
import pandas as pd
import argparse
import torch
import difflib
import nltk
import regex as re
import numpy as np
import MeCab
import pickle
import re
import os


from random import sample
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names, get_dataset_config_names

In [2]:
lang = 'de'

if lang == 'zh':
    dataset_name = "Paul/hatecheck-mandarin"
elif lang == 'es': # gender
    dataset_name = "Paul/hatecheck-spanish" 
elif lang == 'pt': #
    dataset_name = "Paul/hatecheck-portuguese"
elif lang == 'de':
    dataset_name = "Paul/hatecheck-german"
elif lang == 'it':
    dataset_name = "Paul/hatecheck-italian"
elif lang == 'ar':
    dataset_name = "Paul/hatecheck-arabic"
elif lang == 'du':
    dataset_name = "Paul/hatecheck-dutch"
elif lang == 'fr':
    dataset_name = "Paul/hatecheck-french"
elif lang == 'po':
    dataset_name = "Paul/hatecheck-polish"
elif lang == 'hi':
    dataset_name = "Paul/hatecheck-hindi"

else: dataset_name = "Paul/hatecheck"

In [3]:
dataset = load_dataset(dataset_name, split="test")  #, split="test"
dataset[0]

Found cached dataset csv (/home/cass/.cache/huggingface/datasets/Paul___csv/Paul--hatecheck-german-60d44203c479e812/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


{'mhc_case_id': 'german-1',
 'functionality': 'derog_neg_emote_h',
 'test_case': 'Ich hasse Frauen.',
 'label_gold': 'hateful',
 'target_ident': 'women',
 'ref_case_id': None,
 'ref_templ_id': None,
 'templ_id': 1,
 'case_templ': 'Ich hasse [IDENT_P].',
 'gender_male': None,
 'gender_female': None,
 'label_annotated': "['hateful', 'hateful', 'hateful']",
 'label_annotated_maj': 'hateful',
 'disagreement_in_case': False,
 'disagreement_in_template': False}

In [4]:
from deep_translator import GoogleTranslator, MicrosoftTranslator, DeeplTranslator

In [5]:
case_templ_list = []
non_case_templ_list = []
#hate_idt_list = []
idt_list = []

for i in dataset:
    if i['label_gold'] == 'hateful':
        idt_list.append(i['target_ident'])
        #hate_idt_list.append(i['test_case'])
        if lang == 'es' or lang == 'it':
            case_templ_list.append(i['gender_female'])
            case_templ_list.append(i['gender_male'])
        else: case_templ_list.append(i['case_templ'])

    ## non hate
    else:
        
        if i['case_templ'] != None:
            if lang == 'en': 
                if '[IDENTITY_P]' in i['case_templ']: non_case_templ_list.append(i['case_templ'])
                else: pass
            elif '[IDENT_P]' in i['case_templ']: non_case_templ_list.append(i['case_templ'])
        elif i['gender_male'] != None:
            if '[male_IDENT_P]' in i['gender_male']: non_case_templ_list.append(i['gender_male'])
        elif i['gender_female'] != None:
            if '[female_IDENT_P]' in i['gender_female']: non_case_templ_list.append(i['gender_female'])
            # lang == 'es' or lang == 'it':
            # non_case_templ_list.append(i['gender_female'])
            # non_case_templ_list.append(i['gender_male'])
        else: pass
        
non_case_templ_list = list(set(non_case_templ_list))
non_case_templ_list = [x for x in non_case_templ_list if x is not None]
case_templ_list = list(set(case_templ_list))
case_templ_list = [x for x in case_templ_list if x is not None]
idt_list = list(set(idt_list))
idt_list = [x for x in idt_list if x is not None]

print(len(non_case_templ_list), len(case_templ_list))
corpus_num = min(len(non_case_templ_list), len(case_templ_list))
idt_list

84 301


['women',
 'disabled people',
 'trans people',
 'refugees',
 'jews',
 'gay people',
 'black people']

In [6]:
non_case_templ_list

['Du und alle anderen [IDENT_P] sind ein Segen für unsere Gesellschaft.',
 'Aussagen wie "[IDENT_P] sind Abschaum" sind zutiefst verletzend.',
 'Alle [IDENT_P] wie du sind großartig.',
 'Ich werde Beiträge wie "[IDENT_P] sind absolut verachtenswert" in meiner Timeline nicht dulden.',
 'Zu schreiben, dass "alle [IDENT_P] krasse Drecksäcke sind", wirft kein gutes Licht auf Sie.',
 'Du kannst doch nicht ernsthaft glauben, dass "[IDENT_P] keine richtigen Menschen sind".',
 'Aussagen wie "Fick alle [IDENT_P]" zeigen, was für ein schrecklicher Mensch du bist.',
 '[IDENT_P], die das Sagen haben, sind eine Bereicherung für unser Land.',
 'In einer gerechten Welt würden [IDENT_P] alle geehrt werden.',
 'An meiner Schule gibt es sehr wenige [IDENT_P].',
 'Wer "Tod allen [IDENT_P]" wünscht, sollte in den Knast.',
 'Die Leben von [IDENT_P] sind so viel wert.',
 'Wir sollten zu diesen [IDENT_P] aufschauen.',
 '[IDENT_P] haben es nicht verdient, so gefoltert zu werden.',
 '[IDENT_P] in der Regierung

In [7]:
translated_idt_list = []
for idt in idt_list:
    print(idt)
    if lang == 'zh': 
        pass
    else: 
        if lang == 'du': translated_idt = GoogleTranslator(source='auto', target='nl').translate(idt) 
        elif lang == 'po': translated_idt = GoogleTranslator(source='auto', target='pl').translate(idt) 
        else: translated_idt = GoogleTranslator(source='auto', target=lang).translate(idt) 
        print(translated_idt)
        translated_idt_list.append(translated_idt)

if lang == 'zh':
    translated_idt_list = ['穆斯林', '残障人', '变性人', '女人', '黑人', '同性恋', '外国人']
elif lang == 'hi':
    # translated_idt_list = ['मुसलमानों','काले लोग','औरत','निचली जाति','उत्तर-पूर्वी भारतीय','हिज़डा','समलैंगिक लोग'] [Musalmaanon(Musalmanon, Musalmanon), Musalmaano, kaale log, kale log, aurat(aurt), nichli jaati, nichli jati, uttar-poorvi bhartiya(bhaartiya, bhartya), utar-purvi, uttar-poorvi, hijada(hijda), samlaingik log]
    translated_idt_list = ['Musalmaanon', 'kaale log', 'aurat', 'nichli jaati', 'uttar-poorvi bhartiya', 'hijada', 'samlaingik log']

translated_idt_list

women
Frauen
disabled people
Menschen mit Behinderung
trans people
Transmenschen
refugees
Flüchtlinge
jews
Juden
gay people
schwule Menschen
black people
Schwarze


['Frauen',
 'Menschen mit Behinderung',
 'Transmenschen',
 'Flüchtlinge',
 'Juden',
 'schwule Menschen',
 'Schwarze']

In [8]:
case_templ_list = case_templ_list[:corpus_num]
non_case_templ_list = non_case_templ_list[:corpus_num]

total = len(idt_list)*corpus_num
print("==>> each corpus length : ", total)

==>> each corpus length :  588


In [9]:
hate_idt = []
nonhate_idt = []

for idt in translated_idt_list:
    for case_templ in case_templ_list:
        input = re.sub("\[[^\]]*\]", idt, case_templ)
        hate_idt.append(input)
    for non_case_templ in non_case_templ_list:
        input = re.sub("\[[^\]]*\]", idt, non_case_templ)
        nonhate_idt.append(input)

print(len(hate_idt), len(nonhate_idt))
print(hate_idt)

588 588
['Frauen, die das Sagen haben, sind eine Gefahr für unser Land.', 'Es sollte offensichtlich sein, dass Frauen keine Menschen sind.', 'Ihr Frauen seid auch nur Müll.', 'Frauen wie du sollten aufhören, sich zu beschweren. Ihr verdient es zu leiden.', 'Hier Frauen zu sehen, macht mich so wütend.', 'Wir sollten sie einfach erschießen. Sie sind schließlich nur Frauen.', 'Ich glaube, dass es typisch für Frauen ist so dumm zu sein.', 'Wann wird den Leuten endlich klar, dass es in der Natur von Frauen liegt, nervige Trottel zu sein?', 'Ohne Frauen wie dich wäre die Welt eine bessere.', 'Schon wieder diese Frauen.', 'Diese Frauen sehen vielleicht aus wie Menschen, aber sind eigentlich nichts als Parasiten.', 'Frauen gehören nicht auf diese Erde.', 'Eure Leben sind mir einen Scheiß wert. Ihr seid nur Frauen.', 'Dieser Affe ist schlauer als all ihr Frauen zusammen.', 'Frauen wie ihr seid widerwärtig.', 'Ihr Frauen seid alle Lügner.', 'Es liegt in der Natur von Frauen, unzivilisiert zu sei

In [10]:
with open(f"./hate/{lang}/hate_idt.json", 'w') as f:
    json.dump(hate_idt, f, indent=4, ensure_ascii=False) 

with open(f"./hate/{lang}/nonhate_idt.json", 'w') as f:
    json.dump(nonhate_idt, f, indent=4, ensure_ascii=False) 



'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'ilocano': 'ilo', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk', 'khmer': 'km', 'kinyarwanda': 'rw', 'konkani': 'gom', 'korean': 'ko', 'krio': 'kri', 'kurdish (kurmanji)': 'ku', 'kurdish (sorani)': 'ckb', 'kyrgyz': 'ky', 'lao': 'lo', 'latin': 'la', 'latvian': 'lv', 'lingala': 'ln', 'lithuanian': 'lt', 'luganda': 'lg', 'luxembourgish': 'lb', 'macedonian': 'mk', 'maithili': 'mai', 'malagasy': 'mg', 'malay': 'ms', 'malayalam': 'ml', 'maltese': 'mt', 'maori': 'mi', 'marathi': 'mr', 'meiteilon (manipuri)': 'mni-Mtei', 'mizo': 'lus', 'mongolian': 'mn', 'myanmar': 'my', 'nepali': 'ne', 'norwegian': 'no', 'odia (oriya)': 'or', 'oromo': 'om', 'pashto': 'ps', 'persian': 'fa', 'polish': 'pl', 'portuguese': 'pt', 'punjabi': 'pa', 'quechua': 'qu', 'romanian': 'ro', 'russian': 'ru', 'samoan': 'sm', 'sanskrit': 'sa', 'scots gaelic': 'gd', 'sepedi': 'nso', 'serbian': 'sr', 'sesotho': 'st', 'shona': 'sn', 'sindhi': 'sd', 'sinhala': 'si', 'slovak': 'sk', 'slovenian': 'sl', 'somali': 'so', 'spanish': 'es', 'sundanese': 'su', 'swahili': 'sw', 'swedish': 'sv', 'tajik': 'tg', 'tamil': 'ta', 'tatar': 'tt', 'telugu': 'te', 'thai': 'th', 'tigrinya': 'ti', 'tsonga': 'ts', 'turkish': 'tr', 'turkmen': 'tk', 'twi': 'ak', 'ukrainian': 'uk', 'urdu': 'ur', 'uyghur': 'ug', 'uzbek': 'uz', 'vietnamese': 'vi', 'welsh': 'cy', 'xhosa': 'xh', 'yiddish': 'yi', 'yoruba': 'yo', 'zulu': 'zu'

In [11]:
with open('idt_in_DiffLanB.txt', 'a') as writer:
    writer.write('\n')
    writer.write('\n')
    writer.write(lang)
    writer.write('\n')
    writer.write(str(idt_list))
    writer.write('\n')
    writer.write(str(translated_idt_list))